In [1]:
import numpy as np
import pandas as pd

Tensor Elements Order for Inference:

(Left Stick, Right Stick, L2, R2, Triangle, Circle, Cross, Square, North, East, South, West, L1, R1, L2, R2, L3, R3,
       'Byte 16 Gyro X Raw 1', 'Byte 17 Gyro X Raw 2', 'Byte 18 Gyro Y Raw 1',
       'Byte 19 Gyro Y Raw 2', 'Byte 20 Gyro Z Raw 1', 'Byte 21 Gyro Z Raw 2',
       'Byte 22 Accel X Raw 1', 'Byte 23 Accel X Raw 2',
       'Byte 24 Accel Y Raw 1', 'Byte 25 Accel Y Raw 2',
       'Byte 26 Accel Z Raw 1', 'Byte 27 Accel Z Raw 2')

neutral: 0x8, N: 0x0, NE: 0x1, E: 0x2, SE: 0x3, S: 0x4, SW: 0x5, W: 0x6, NW: 0x7
byte 5, bit 4: Button / 0x01 - 1 bit - Square button
byte 5, bit 5: Button / 0x02 - 1 bit - Cross button
byte 5, bit 6: Button / 0x03 - 1 bit - Circle button
byte 5, bit 7: Button / 0x04 - 1 bit - Triangle button

**Resources:**
- [Nondebug](https://github.com/nondebug/dualsense)
- [Dualsense USB Descriptor Report](https://github.com/nondebug/dualsense/blob/main/report-descriptor-bluetooth.txt)
- [DSRemap](https://dsremap.readthedocs.io/en/latest/reverse.html)



In [74]:
data_directory = './data/'
df = pd.read_csv(data_directory + 'dual_sense_data_high_intensity.csv', index_col=0)

def byte_to_bits(input_decimal):
    byte = format(input_decimal, '08b')
    bits = [byte[0], byte[1], byte[2], byte[3], byte[4], byte[5], byte[6], byte[7]]
    num = [int(bit, 2) for bit in bits]
    return num

def bytes_to_bits(input_decimals):
    bits = np.zeros((input_decimals.shape[0], 8), dtype=np.int8)
    for i, hex_ in enumerate(input_decimals):
        byte = format(int(hex_), '08b')
        byte_split = [byte[0], byte[1], byte[2], byte[3], byte[4], byte[5], byte[6], byte[7]]
        bits[i] = [int(bit, 2) for bit in byte_split]
    return bits

def to_unipolar(values):
    return (values / 255) * 2 - 1

def col_match_drop(dataframe, words_to_find):
    indeces_to_drop = []
    for i, col in enumerate(dataframe.columns):
        for word in words_to_find:
            if (word in col.lower()):
                indeces_to_drop.append(i)
                
    dataframe = dataframe.drop(dataframe.columns[indeces_to_drop],axis = 1)
    return dataframe

def insert_dataframe_columns(destination, source, loc):
    front = destination.drop(destination.columns[loc:],axis = 1)
    back = destination.drop(destination.columns[:loc],axis = 1)
    new_frame = pd.concat([source, back], axis=1)
    new_frame = pd.concat([front, new_frame], axis=1)
    return new_frame

def bytes_to_unipolar(bytesA=[], bytesB=[]):
    vectors = [[0 for i in range(2)] for i in range(bytesA.shape[0])]
    for byte1, byte2 in zip(bytesA, bytesB):
        new_byte = (byte2 & 0xFF) << 8 | (byte1 & 0xFF)
        return (new_byte / 65535) * 2 - 1
    return vectors

In [3]:
# Drop unimportant features.
df = col_match_drop(df, ['report', 'unknown', 'timestamp', 'vendor', 'touchpad', 'multiplier'])
# Decimal binary representations to unipolar
df.iloc[:, 0:4] = to_unipolar(df.iloc[:, 0:4].to_numpy())
df.iloc[:, 8:20] = to_unipolar(df.iloc[:, 8:20].to_numpy())
# Split byte - Arrows and Buttons
btns = bytes_to_bits(df.iloc[:, 7])

In [4]:
# Face buttons use 1 bit each, so we can use them as-is.
face_labels = ['Triangle', 'Circle', 'Cross', 'Square']
face_buttons_frame = pd.DataFrame(btns[:, :4],columns=face_labels)

# D-Pad uses uses the first nibble to describe 9 states
# States: 8=neutral, 0=N, 1=NE, 2=E, 3=SE, 4=S, 5=SW, 6=W, 7=NW
# Game engines are only interested in N, E, S, W, so we filter the others out.
dpad_labels = ['North', 'East', 'South', 'West']
bitmask = int('00000110', 2) # Filter out odd numbers between (0, 8]. 
num_check = [0, 2, 4, 6] # dpad messages in base10 to check for, represents N, E, S, and W.
dpad_activity = [[0 for i in range(4)] for i in range(df.shape[0])] # Store results.
for i, dpad_byte in enumerate(df.iloc[:, 7]):
    # Convert base10 byte to base2
    dpad_action = format(int(dpad_byte), '08b')
    filtered_byte = bin(bitmask & int(dpad_action))
    #print(format(int(filtered_byte, 2), '08b'))
    for j, n in enumerate(num_check):
        if(int(filtered_byte, 2) == num_check[j]):
            dpad_activity[i][j] = 1

dpad_frame = pd.DataFrame(dpad_activity, columns=dpad_labels)

In [5]:
# Insert new frames in place of base10 columns.
df = df.drop(df.columns[6],axis = 1)

In [6]:
btn_frame = pd.concat([face_buttons_frame, dpad_frame], axis=1)
df = insert_dataframe_columns(df, btn_frame, 6)

In [7]:
# Bumpers use 1 bit each, so we can use them as-is.
triggers_labels = ['L1', 'R1', 'L2', 'R2', 'Create', 'Options', 'L3', 'R3']
df = df.reset_index(drop=True)
triggers_bytes = bytes_to_bits(df.iloc[:, 14])

In [8]:
triggers_frame = pd.DataFrame(triggers_bytes, columns=triggers_labels)
triggers_frame = triggers_frame.drop(triggers_frame.columns[[4,5]],axis = 1) # Remove buttons not used for gameplay

In [9]:
df = df.drop(df.columns[14],axis = 1)
df = insert_dataframe_columns(df, triggers_frame, 14)

In [10]:
df.columns

Index(['Byte 1 Left Stick X', 'Byte 2 Left Stick Y', 'Byte 3 Right Stick X',
       'Byte 4 Right Stick Y', 'Byte 5 L2 Trigger Axis',
       'Byte 6 R2 Trigger Axis', 'Triangle', 'Circle', 'Cross', 'Square',
       'North', 'East', 'South', 'West', 'L1', 'R1', 'L2', 'R2', 'L3', 'R3',
       'Byte 16 Gyro X Raw 1', 'Byte 17 Gyro X Raw 2', 'Byte 18 Gyro Y Raw 1',
       'Byte 19 Gyro Y Raw 2', 'Byte 20 Gyro Z Raw 1', 'Byte 21 Gyro Z Raw 2',
       'Byte 22 Accel X Raw 1', 'Byte 23 Accel X Raw 2',
       'Byte 24 Accel Y Raw 1', 'Byte 25 Accel Y Raw 2',
       'Byte 26 Accel Z Raw 1', 'Byte 27 Accel Z Raw 2'],
      dtype='object')

In [84]:
a = pd.DataFrame([1,2,3,4,3,4])


In [87]:
vectors = [[0 for i in range(2)] for i in range(a.shape[0])]
vectors

[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]

In [75]:
bytes_to_unipolar(128, 255)

0.9961242084382391

In [38]:
decA = 255
decB = 255

a = int('11111111', 2)
b = int('11111111', 2)
format((b << 8),'b')

'1111111100000000'

In [39]:
format(decB, 'b')

'11111111'

In [40]:
((a & 0xFF) << 8) | (b & 0xFF)

65535

In [29]:
gyroX = (((int)decB) << 8) | decA;
gyroY = (((int)_buff[3]) << 8) | _buff[2];
gyroZ = (((int)_buff[5]) << 8) | _buff[4];

SyntaxError: invalid syntax. Perhaps you forgot a comma? (4207801779.py, line 1)